#SIG sujet n°1 : Réaliser une carte des villes du monde étant dans la zone d'impact de séismes ayant une amplitude supérieure à 6,1


L'objectif de ce projet est de créer une carte des villes du monde situées dans des zones susceptibles d'être impactées par des séismes d'une amplitude supérieure à 6,1 sur l'échelle de Richter, ces séismes ayant une portée d'environ 100 kilomètres. Cette carte identifie les villes exposées à un risque sismique significatif, ce qui facilite la mise en place de mesures visant à atténuer ce risque.

#Sources des données

US Geological Survey :
https://earthquake.usgs.gov/earthquakes/search/
* Seismes avec une magnitude supérieure ou égale à 6.1, répertoriés depuis 1900

SimpleMaps : https://simplemaps.com/data/world-cities

In [93]:
!pip install geodatasets
!pip install mapclassify

import pandas as pd
import geopandas as gpd
import geodatasets
import folium

from shapely.geometry import Point
from geopy.distance import geodesic

#Nettoyage et visualisation des données

In [94]:
catalog = gpd.read_file('/content/earthquakes.json')

In [95]:
catalog.crs

<Geographic 3D CRS: EPSG:4979>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
- h[up]: Ellipsoidal height (metre)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [96]:
catalog.columns

Index(['id', 'mag', 'place', 'time', 'updated', 'tz', 'url', 'detail', 'felt',
       'cdi', 'mmi', 'alert', 'status', 'tsunami', 'sig', 'net', 'code', 'ids',
       'sources', 'types', 'nst', 'dmin', 'rms', 'gap', 'magType', 'type',
       'title', 'geometry'],
      dtype='object')

In [97]:
catalog.drop(['id', 'time', 'updated', 'tz', 'url', 'detail', 'felt', 'cdi',  'alert', 'status', 'tsunami', 'sig', 'net', 'code', 'ids',
       'sources', 'types', 'nst', 'dmin', 'rms', 'gap', 'magType', 'type', 'title', 'mmi'], axis = 1, inplace=True)

catalog.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 11173 entries, 0 to 11172
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   mag       11173 non-null  float64 
 1   place     11073 non-null  object  
 2   geometry  11138 non-null  geometry
dtypes: float64(1), geometry(1), object(1)
memory usage: 262.0+ KB


##Visualisation des séismes

In [98]:
catalog = catalog.dropna()


m = catalog.explore(column = catalog["mag"],
                        cmap = "RdYlGn_r",
                        legend = True,
                        name="Magnitude")

folium.LayerControl().add_to(m)
m

Output hidden; open in https://colab.research.google.com to view.

##Visualisation des villes

In [99]:
cities = pd.read_csv('/content/worldcities.csv')

In [100]:
cities.drop(['city_ascii', 'iso2', 'iso3',
       'admin_name', 'capital', 'id'], axis=1, inplace =True)

In [101]:
cities.columns

Index(['city', 'lat', 'lng', 'country', 'population'], dtype='object')

In [102]:
geometry = [Point(lon, lat) for lon, lat in zip(cities['lng'], cities['lat'])]
cities = gpd.GeoDataFrame(cities, geometry=geometry, crs=4979) #crs="EPSG:3857"


In [103]:
cities

,city,lat,lng,country,population,geometry
0,Tokyo,35.6897,139.6922,Japan,37732000.0,POINT (139.69220 35.68970)
1,Jakarta,-6.1750,106.8275,Indonesia,33756000.0,POINT (106.82750 -6.17500)
2,Delhi,28.6100,77.2300,India,32226000.0,POINT (77.23000 28.61000)
3,Guangzhou,23.1300,113.2600,China,26940000.0,POINT (113.26000 23.13000)
4,Mumbai,19.0761,72.8775,India,24973000.0,POINT (72.87750 19.07610)
...,...,...,...,...,...,...
44686,Numto,63.6667,71.3333,Russia,10.0,POINT (71.33330 63.66670)
44687,Nord,81.7166,-17.8000,Greenland,10.0,POINT (-17.80000 81.71660)
44688,Timmiarmiut,62.5333,-42.2167,Greenland,10.0,POINT (-42.21670 62.53330)
44689,San Rafael,-16.7795,-60.6799,Bolivia,NaN,POINT (-60.67990 -16.77950)


In [104]:
cities.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 44691 entries, 0 to 44690
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   city        44691 non-null  object  
 1   lat         44691 non-null  float64 
 2   lng         44691 non-null  float64 
 3   country     44691 non-null  object  
 4   population  44384 non-null  float64 
 5   geometry    44691 non-null  geometry
dtypes: float64(3), geometry(1), object(2)
memory usage: 2.0+ MB


In [105]:
cities = cities.dropna()


m = cities.explore(column = cities["population"],
                        cmap = "RdYlGn_r",
                        legend = True,
                        vmax=40000,
                        name="population")

folium.LayerControl().add_to(m)
m

Output hidden; open in https://colab.research.google.com to view.

#Délimiation de la zone d'impact des séismes

On considérera qu'un séisme d'amplitude 6,1 ou plus a une zone d'impact  de 100 km

In [106]:
catalog.columns

Index(['mag', 'place', 'geometry'], dtype='object')

In [111]:
#Prendre en compte la courbure de la terre lors de la création des buffers

list_buffer = []
for point in catalog.geometry:

  # Define the buffer distance in kilometers (100 kilometers)
  buffer_distance_km = 100

  # Calculate the buffer in degrees of latitude and longitude
  buffer_distance_deg = (buffer_distance_km / geodesic((0, point.y), (0, point.y + 1)).kilometers)

# Create a buffer around the point
  buffered_point = point.buffer(buffer_distance_deg)
  list_buffer.append(buffered_point)

# Print the resulting buffered point
zone_impact = gpd.GeoDataFrame(geometry=list_buffer, crs=catalog.crs)
print(zone_impact)

                                                geometry
0      POLYGON ((147.09652 -5.50660, 147.09219 -5.594...
1      POLYGON ((147.02982 -5.59970, 147.02549 -5.687...
2      POLYGON ((62.80202 34.57430, 62.79769 34.48625...
3      POLYGON ((62.82252 34.61020, 62.81819 34.52215...
4      POLYGON ((140.90292 30.04400, 140.89859 29.955...
...                                                  ...
11033  POLYGON ((101.50632 30.68400, 101.50199 30.595...
11034  POLYGON ((161.17532 52.76300, 161.17099 52.674...
11035  POLYGON ((162.53632 51.42400, 162.53199 51.335...
11036  POLYGON ((24.14732 41.75800, 24.14299 41.66995...
11037  POLYGON ((24.00632 41.80200, 24.00199 41.71395...

[11038 rows x 1 columns]


In [112]:
m=zone_impact.explore()
folium.LayerControl().add_to(m)
m

Output hidden; open in https://colab.research.google.com to view.

In [113]:
zone_impact_dissolve = zone_impact.dissolve()
zone_impact_dissolve.explore()

Output hidden; open in https://colab.research.google.com to view.

#Identification des villes situées dans les zones d'impact

In [114]:
points_in_polys =cities.clip(zone_impact_dissolve)
points_in_polys

,city,lat,lng,country,population,geometry
43752,Te Anau,-45.4167,167.7167,New Zealand,2250.0,POINT (167.71670 -45.41670)
29355,Macetown,-44.8650,168.8190,New Zealand,16600.0,POINT (168.81900 -44.86500)
42366,Wanaka,-44.7000,169.1500,New Zealand,8900.0,POINT (169.15000 -44.70000)
28165,Ashton,-44.0333,171.7667,New Zealand,17700.0,POINT (171.76670 -44.03330)
29491,Rolleston,-43.5833,172.3833,New Zealand,16350.0,POINT (172.38330 -43.58330)
...,...,...,...,...,...,...
18823,Kópavogur,64.1108,-21.9094,Iceland,34140.0,POINT (-21.90940 64.11080)
841,Reykjavík,64.1467,-21.9400,Iceland,135688.0,POINT (-21.94000 64.14670)
43780,Borgarnes,64.5333,-21.9167,Iceland,1962.0,POINT (-21.91670 64.53330)
26923,Akureyri,65.6833,-18.1000,Iceland,18925.0,POINT (-18.10000 65.68330)


In [117]:
m = points_in_polys.explore(column = points_in_polys["population"],
                        cmap = "RdYlGn_r",
                        legend = True,
                        vmax=40000,
                        name="Villes situées dans des zones à risque sismique")

zone_impact_dissolve.explore(m=m, name='zones à risque sismique')

folium.LayerControl().add_to(m)
m

Output hidden; open in https://colab.research.google.com to view.

#Visualisation et interprétation des résultats

In [118]:
plates = gpd.read_file('/content/PB2002_plates.json')

In [158]:
plates.crs #EPSG:4326

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [ ]:
plates.to_crs(4979)

In [164]:
m = points_in_polys.explore(column = points_in_polys["population"],
                        cmap = "RdYlGn_r",
                        legend = True,
                        vmax=40000,
                        name="Villes situées dans des zones à risque sismique",  legend_kwds={"label": "Population"})

zone_impact_dissolve.explore(m=m, name='zones à risque sismique', popup=False)

plates.explore(m=m, column='PlateName', tooltip = ["Code", "PlateName"],
                tooltip_kwds = dict(aliases = ["Abréviation", "Nom de la plaque"]), name='Plaques tectoniques',
               style_kwds = dict(color="black", fillOpacity=0.4))

folium.LayerControl().add_to(m)
m
from google.colab import files #Permet d'exporter la carte finale sur votre pc directement
m.save("PEYROT_Leane_carte_finale.html")   #Sauvegarde TEMPORAIRE dans l'environnement Colab (icone dossier à gauche)
files.download("PEYROT_Leane_carte_finale.html") # Télécharge le fichier sur votre pc

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Les villes situées sur la zone d'impact de séismes ayant une amplitude supérieure à 6,1 sont majoritairement localisées aux limites des plaques tectoniques, où les contraintes tectoniques sont importantes.